In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from resnet18_32x32 import ResNet18_32x32
import torch.nn.functional as F
from scipy.stats import wasserstein_distance

from torchvision import transforms

from sklearn.manifold import TSNE
import numpy as np
from scipy.special import kl_div

In [2]:
# 加载训练好的模型
model = ResNet18_32x32()
model.load_state_dict(torch.load('./resnet/model.pth'))
output = "./output_prob"


# 加载CIFAR-10数据集
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=2)



# 使用模型预测类别概率分布
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


Files already downloaded and verified


In [5]:
def compute_js_divergence(p, q):
    m = 0.5 * (p + q)  # 计算均值分布
    js_div = 0.5 * np.sum(kl_div(p, m)) + 0.5 * np.sum(kl_div(q, m))
    return js_div

def compute_average_js_divergence(real_distributions, generated_distributions):
    num_distributions = len(real_distributions)
    total_js_div = 0.0

    for i in range(num_distributions):
        real_distribution = real_distributions[i]
        generated_distribution = generated_distributions[i]
        js_div = compute_js_divergence(real_distribution, generated_distribution)
        total_js_div += js_div

    average_js_div = total_js_div / num_distributions
    return average_js_div

In [9]:
test_probabilities = []
with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device)
        outputs = model(images)
        probabilities = F.softmax(outputs, dim=1)
        test_probabilities.append(probabilities.cpu().numpy()[0])

test_probabilities = np.array(test_probabilities)

In [10]:
# 加载训练集和测试集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=False, num_workers=2)

# 使用模型预测类别概率分布
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Files already downloaded and verified


In [11]:
train_probabilities = []
count = 0

with torch.no_grad():
    for images, labels in trainloader:
        images = images.to(device)
        outputs = model(images)
        probabilities = F.softmax(outputs, dim=1)
        train_probabilities.append(probabilities.cpu().numpy()[0])
        
        count += images.size(0)
        if count >= 10000:
            break

train_probabilities = np.array(train_probabilities)

In [15]:
latent_dim = 100
num_classes = 10
lr = 0.0002
batch_size = 1
num_epochs = 20
num_samples = 10000    
    
class Generator(nn.Module):
    def __init__(self, latent_dim, num_classes):
        super(Generator, self).__init__()
        self.label_embed = nn.Embedding(num_classes, latent_dim)
        self.generator = nn.Sequential(
            nn.Linear(latent_dim * 2, 128),  # 将噪音维度加入生成器输入维度
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, num_classes),
            nn.Softmax(dim=1)
        )
    
    def forward(self, labels, noise):
        gen_input = self.label_embed(labels)
        gen_input_with_noise = torch.cat((gen_input, noise), dim=1)  # 将噪音和标签嵌入向量连接起来
        class_probs = self.generator(gen_input_with_noise)
        return class_probs

In [22]:
# 加载生成模型
generator = Generator(latent_dim, num_classes).to(device)
generator.load_state_dict(torch.load('./output/generator_100+20.pth'))  # 将X替换为你想要加载的模型的epoch数
generated_probs = []
# 测试生成器



generator.eval()

Generator(
  (label_embed): Embedding(10, 100)
  (generator): Sequential(
    (0): Linear(in_features=200, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): Linear(in_features=512, out_features=10, bias=True)
    (7): Softmax(dim=1)
  )
)

In [23]:
generator_probabilities = []
with torch.no_grad():
    for _ in range(num_samples):
        labels = torch.randint(0, num_classes, (batch_size,), device=device)
        noise = torch.randn(batch_size, latent_dim, device=device)
        generated_probabilities = generator(labels, noise)
        generator_probabilities.append(generated_probabilities.cpu().numpy()[0])

generator_probabilities = np.array(generator_probabilities)

In [24]:
# 计算平均JS散度
average_js_distance = compute_average_js_divergence(test_probabilities, generator_probabilities)
print("平均JS散度:", average_js_distance)

平均JS散度: 0.4951479776121676
